In [1]:
# Specify credentials file
CLIENT_SECRETS_FILE = "client_secret.json"

In [2]:
# Restrict scope to YouTube only
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [3]:
# Service to interect with API
import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


In [5]:
import os
if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=590083949853-sjddsq4nsq6j3dbfi7ghft05e2k62n5r.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=RzmahqRCsw593tkmeSyCXFdDAbrLVo&prompt=consent&access_type=offline
Enter the authorization code: 4/wAFNcMfwOXMcRGsF2qNA10VidjfAD7p26ev0yB4PaXEj7wwf2M9MlcU


In [7]:
# Cache credentials
import os
import pickle
import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


In [8]:
# Search videos by keyword
keyword = input('Enter a keyword: ')


Enter a keyword: superwoman


In [10]:
def search_videos_by_keyword(service, **kwargs):
    results = service.search().list(**kwargs).execute()
    for item in results['items']:
        print('%s - %s' % (item['snippet']['title'], item['id']['videoId']))

keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')


Enter a keyword: superwoman
#LillyLIVE: Third Wheel Edition (ft. Alex Wassabi and LaurDIY) - 7As3V3QlfI4
#AskSuperwomanLIVE: CUT THE BS (ft. fouseyTUBE) - FM2Yj7mU21U
#AskSuperwomanLIVE from Rio! (08/15/16) - NA1lQU0hh4g
Story Time with Lilly: LIVE - _eQuL6o_l2w
#AskSuperwomanLIVE ft. My Parents - PdHHDYncH68


In [12]:
def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()

    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])

        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break

    return final_results

def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    for item in results:
        print('%s - %s' % (item['snippet']['title'], item['id']['videoId']))


keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')


Enter a keyword: superwoman
#LillyLIVE: Third Wheel Edition (ft. Alex Wassabi and LaurDIY) - 7As3V3QlfI4
#AskSuperwomanLIVE: CUT THE BS (ft. fouseyTUBE) - FM2Yj7mU21U
#AskSuperwomanLIVE from Rio! (08/15/16) - NA1lQU0hh4g
Story Time with Lilly: LIVE - _eQuL6o_l2w
#AskSuperwomanLIVE ft. My Parents - PdHHDYncH68
Ask Superwoman LIVE! - nU_rfS52x-A
#AskSuperwomanLIVE - TRUTH OR DARE - fiTLFz5ArYI
Studio Salon: Black Macho and the Myth of the Superwoman - LSEUHDer6pk
Superwoman Lives! - U-_jeFov6Jw
#AskSuperwomanLIVE - 01/14/17 - 64mNTihmyXQ
#AskSuperwomanLIVE (01/14/15) - S107HCcZYKA
Superwoman (WL &amp; PL Combined) - 5 Bar Showdown - RLSUi4WYfOI
Superwoman update! - nQjY47lnAIg
VOR 🔥🔥🔥🤯🤯🤯TOPIC: “Phuck-it, I’ll Do it.” “Superwoman Syndrome.” - vrNR9-WNh7U
SuperWoman ГАЛА-ШОУ 2019 Петрозаводск - VNhyNCqo9uQ


In [13]:
# Get comments
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()

    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments


In [14]:
def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        
        print(comments)


In [15]:
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')

Enter a keyword: superwoman
["who's here after #LAUREX broke up :(", '2019? 😭', 'Missing this...', '2019 Anyone?', 'is anyone watching in 2019???? :/\n\nps-im a small youtuber', 'Alex we all know that you had a teddy bear when you were younger', '“Streamed two years ago” whaaaat I feel like I just watched this live like last month😧', '....anyone here after their breakup...', '#RELATABLE at the Bollywood part 😝😂☺️ 31:06', '15:30 Stress Fart?!!! I knew it was coming...I’m Lilly 😂', 'Who is max', '2019.....too bad the split up..', 'But they broke up alreday😭😭😭', "Lilly: ok so I'm going to change this. What kind of person do you see me dating \nLauren: (clap) oh I've thought about this before 😃\nLilly: OH...😐", 'I would melt into a puddle and the puddle would spell no', 'I watched that Bollywood movie', "I cant believe they're not together anymore...", 'Who came after they broke up 😭😖😩👎🏼😟☹️🙁😕😫🥺😐😞😢😑😮😔😣😳', 'Actually the name superwoman is a villain in the comics', 'I have the walking fart to

KeyboardInterrupt: 

In [16]:
def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    final_result = []
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        final_result.extend([(video_id, title, comment) for comment in comments])


In [17]:
import csv

def write_to_csv(comments):
    with open('comments.csv', 'w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['Video ID', 'Title', 'Comment'])
        for row in comments:
            comments_writer.writerow(list(row))


In [18]:
import csv
import os

import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()
i=0
    while i<3:
        i=i+1
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments


def write_to_csv(comments):
    with open('comments.csv', 'w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['Video ID', 'Title', 'Comment'])
        for row in comments:
            # convert the tuple to a list and write to the output file
            comments_writer.writerow(list(row))


def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()

    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])

        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break

    return final_results


def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    final_result = []
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        # make a tuple consisting of the video id, title, comment and add the result to 
        # the final list
        final_result.extend([(video_id, title, comment) for comment in comments]) 

    write_to_csv(final_result)


if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()
    keyword = input('Enter a keyword: ')
    search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=590083949853-sjddsq4nsq6j3dbfi7ghft05e2k62n5r.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=L5m05zbDOZ4rwgGnTujtrbkog5lthL&prompt=consent&access_type=offline
Enter the authorization code: 4/wAHteCo5F1f_CQOM7H6puKOztiObZjELP1sY3bJob8NXIoouPBorcJg
Enter a keyword: superwoman


HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=LSEUHDer6pk&textFormat=plainText&alt=json returned "The video identified by the <code><a href="/youtube/v3/docs/commentThreads/list#videoId">videoId</a></code> parameter has disabled comments.">

In [11]:
import os
import pickle
import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [12]:
import googleapiclient.discovery
def get_video_comments(service, **kwargs):
    comments = []
    results = service.comment().list(**kwargs).execute()
    i=0
    while i<3:
        i=i+1
        print(i)
#         for item in results['items']:
#             comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
#             comments.append(comment)

#         # Check if another page exists
#         if 'nextPageToken' in results:
#             kwargs['pageToken'] = results['nextPageToken']
#             results = service.comment().list(**kwargs).execute()
#         else:
#             break
#     request = service.comments().list(
#         part="snippet",
#         parentId="UgzDE2tasfmrYLyNkGt4AaABAg"
#     )
#     response = request.execute()

#     print(response)

#     request = service.videos().list(
#         part="statistics",
#         id="Ks-_Mh1QhMc"
#         )
#     response = request.execute()

#     print(response)
    
    

#     return comments

In [13]:
if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()
#     keyword = input('Enter a keyword: ')
#     search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')
    get_video_comments(service,)


1
2
3
